In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1831831,2021-11-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1831832,2021-11-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1831833,2021-11-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1831834,2021-11-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
47611,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47613,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47615,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47617,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
47619,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1114499,2021-11-10,Arkansas,Arkansas,5001,3513,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1114501,2021-11-11,Arkansas,Arkansas,5001,3516,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1114503,2021-11-12,Arkansas,Arkansas,5001,3517,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1114505,2021-11-13,Arkansas,Arkansas,5001,3519,60.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1831831,2021-11-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1831832,2021-11-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1831833,2021-11-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1831834,2021-11-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-11-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
663,2021-11-14,Snohomish,Washington,68502,773.00,53061,WA,County,822083,8332.74,94.03
1324,2021-11-14,Cook,Illinois,654197,11923.00,17031,IL,County,5150233,12702.28,231.50
1984,2021-11-14,Orange,California,330093,5643.00,6059,CA,County,3175692,10394.36,177.69
2643,2021-11-14,Maricopa,Arizona,767759,12459.00,4013,AZ,County,4485414,17116.79,277.77
3302,2021-11-14,Los Angeles,California,1512028,26862.00,6037,CA,County,10039107,15061.38,267.57
...,...,...,...,...,...,...,...,...,...,...,...
1830367,2021-11-14,Wheeler,Oregon,108,1.00,41069,OR,County,1332,8108.11,75.08
1830765,2021-11-14,King,Texas,20,0.00,48269,TX,County,272,7352.94,0.00
1831132,2021-11-14,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10
1831495,2021-11-14,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
663,2021-11-14,Snohomish,Washington,68502,773.00,53061,WA,County,822083,8332.74,94.03,94.03,8332.74
1324,2021-11-14,Cook,Illinois,654197,11923.00,17031,IL,County,5150233,12702.28,231.50,231.50,12702.28
1984,2021-11-14,Orange,California,330093,5643.00,6059,CA,County,3175692,10394.36,177.69,177.69,10394.36
2643,2021-11-14,Maricopa,Arizona,767759,12459.00,4013,AZ,County,4485414,17116.79,277.77,277.77,17116.79
3302,2021-11-14,Los Angeles,California,1512028,26862.00,6037,CA,County,10039107,15061.38,267.57,267.57,15061.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830367,2021-11-14,Wheeler,Oregon,108,1.00,41069,OR,County,1332,8108.11,75.08,75.08,8108.11
1830765,2021-11-14,King,Texas,20,0.00,48269,TX,County,272,7352.94,0.00,0.00,7352.94
1831132,2021-11-14,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10,229.10,7101.95
1831495,2021-11-14,Loving,Texas,8,0.00,48301,TX,County,169,4733.73,0.00,0.00,4733.73


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1165760,2021-11-14,Hale,Alabama,3155,88.00,1065,AL,County,14651,21534.37,600.64,600.64,21534.37,1
1269805,2021-11-14,Clarke,Alabama,4822,86.00,1025,AL,County,23622,20413.17,364.07,364.07,20413.17,2
983654,2021-11-14,Winston,Alabama,4756,106.00,1133,AL,County,23629,20127.81,448.60,448.60,20127.81,3
676296,2021-11-14,Franklin,Alabama,6226,108.00,1059,AL,County,31362,19852.05,344.37,344.37,19852.05,4
329044,2021-11-14,Calhoun,Alabama,22481,515.00,1015,AL,County,113605,19788.74,453.33,453.33,19788.74,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112122,2021-11-14,Converse,Wyoming,2270,39.00,56009,WY,County,13822,16423.09,282.16,282.16,16423.09,19
980664,2021-11-14,Johnson,Wyoming,1271,16.00,56019,WY,County,8445,15050.33,189.46,189.46,15050.33,20
1112719,2021-11-14,Sublette,Wyoming,1438,21.00,56035,WY,County,9831,14627.20,213.61,213.61,14627.20,21
1385247,2021-11-14,Lincoln,Wyoming,2891,23.00,56023,WY,County,19830,14578.92,115.99,115.99,14578.92,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
910581,2021-11-14,Lowndes,Alabama,1779,67.00,1085,AL,County,9726,18291.18,688.88,688.88,18291.18,22,1
1165760,2021-11-14,Hale,Alabama,3155,88.00,1065,AL,County,14651,21534.37,600.64,600.64,21534.37,1,2
385492,2021-11-14,Walker,Alabama,11847,377.00,1127,AL,County,63521,18650.53,593.50,593.50,18650.53,16,3
806187,2021-11-14,Dallas,Alabama,5263,207.00,1047,AL,County,37196,14149.37,556.51,556.51,14149.37,62,4
982458,2021-11-14,Greene,Alabama,1294,45.00,1063,AL,County,8111,15953.64,554.80,554.80,15953.64,49,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738981,2021-11-14,Weston,Wyoming,1175,10.00,56045,WY,County,6927,16962.61,144.36,144.36,16962.61,16,19
1350378,2021-11-14,Uinta,Wyoming,3950,29.00,56041,WY,County,20226,19529.32,143.38,143.38,19529.32,7,20
1385247,2021-11-14,Lincoln,Wyoming,2891,23.00,56023,WY,County,19830,14578.92,115.99,115.99,14578.92,22,21
906987,2021-11-14,Albany,Wyoming,6565,36.00,56001,WY,County,38880,16885.29,92.59,92.59,16885.29,17,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,94.03,8332.74,29,28
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,94.03,8332.74,29,28
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,94.03,8332.74,29,28
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,94.03,8332.74,29,28
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,94.03,8332.74,29,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827589,2021-11-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1827590,2021-11-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1827591,2021-11-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1827592,2021-11-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1160924,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21534.37,2,1,1.00,nan
1160925,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21534.37,2,1,0.00,nan
1160926,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21534.37,2,1,0.00,nan
1160927,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,600.64,21534.37,2,1,0.00,nan
1160928,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,600.64,21534.37,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408731,2021-11-10,Crook,Wyoming,943,20.00,56011,WY,County,7584,12434.07,263.71,263.71,12605.49,10,23,1.00,4.29
1408732,2021-11-11,Crook,Wyoming,943,20.00,56011,WY,County,7584,12434.07,263.71,263.71,12605.49,10,23,0.00,4.07
1408733,2021-11-12,Crook,Wyoming,956,20.00,56011,WY,County,7584,12605.49,263.71,263.71,12605.49,10,23,13.00,4.50
1408734,2021-11-13,Crook,Wyoming,956,20.00,56011,WY,County,7584,12605.49,263.71,263.71,12605.49,10,23,0.00,4.50


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
906935,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18291.18,1,22,1.00,2.21,0.00,nan
906936,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18291.18,1,22,0.00,2.07,0.00,nan
906937,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18291.18,1,22,0.00,1.86,0.00,nan
906938,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18291.18,1,22,0.00,1.50,0.00,nan
906939,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,688.88,18291.18,1,22,0.00,1.43,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381222,2021-11-10,Teton,Wyoming,5286,14.00,56039,WY,County,23464,22528.13,59.67,59.67,22592.06,23,3,5.00,8.43,0.00,0.00
381223,2021-11-11,Teton,Wyoming,5286,14.00,56039,WY,County,23464,22528.13,59.67,59.67,22592.06,23,3,0.00,7.21,0.00,0.00
381224,2021-11-12,Teton,Wyoming,5301,14.00,56039,WY,County,23464,22592.06,59.67,59.67,22592.06,23,3,15.00,6.93,0.00,0.00
381225,2021-11-13,Teton,Wyoming,5301,14.00,56039,WY,County,23464,22592.06,59.67,59.67,22592.06,23,3,0.00,6.93,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-11-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
460604,2021-11-14,Imperial,California,37398,765.00,6025,CA,County,181215,20637.36,422.15,422.15,20637.36,1,3,0.00,66.93,0.00,0.00
228570,2021-11-14,San Bernardino,California,367946,5873.00,6071,CA,County,2180085,16877.60,269.39,269.39,16877.60,2,6,0.00,476.93,0.00,6.86
3302,2021-11-14,Los Angeles,California,1512028,26862.00,6037,CA,County,10039107,15061.38,267.57,267.57,15061.38,3,14,890.00,1296.71,6.00,16.07
101833,2021-11-14,Stanislaus,California,89701,1387.00,6099,CA,County,550660,16289.73,251.88,251.88,16289.73,4,7,0.00,159.64,0.00,2.14
1113245,2021-11-14,Tuolumne,California,7608,135.00,6109,CA,County,54478,13965.27,247.81,247.81,13965.27,5,19,44.00,22.43,1.00,1.00
80925,2021-11-14,San Joaquin,California,105829,1803.00,6077,CA,County,762148,13885.62,236.57,236.57,13885.62,6,21,0.00,150.21,0.00,2.50
809139,2021-11-14,Inyo,California,2473,42.00,6027,CA,County,18039,13709.19,232.83,232.83,13709.19,7,22,0.00,13.43,0.00,0.07
732829,2021-11-14,Merced,California,44566,645.00,6047,CA,County,277680,16049.41,232.28,232.28,16049.41,8,9,0.00,76.71,0.00,3.79
982400,2021-11-14,Kings,California,34573,347.00,6031,CA,County,152940,22605.60,226.89,226.89,22605.60,9,2,0.00,67.64,0.00,0.71
70454,2021-11-14,Shasta,California,25443,406.00,6089,CA,County,180080,14128.72,225.46,225.46,14128.72,10,18,0.00,65.71,0.00,3.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1726657,2021-11-14,Lassen,California,7843,54.00,6035,CA,County,30573,25653.35,176.63,176.63,25653.35,20,1,0.00,12.29,0.00,0.21
982400,2021-11-14,Kings,California,34573,347.00,6031,CA,County,152940,22605.60,226.89,226.89,22605.60,9,2,0.00,67.64,0.00,0.71
460604,2021-11-14,Imperial,California,37398,765.00,6025,CA,County,181215,20637.36,422.15,422.15,20637.36,1,3,0.00,66.93,0.00,0.00
126390,2021-11-14,Tulare,California,84621,1033.00,6107,CA,County,466195,18151.42,221.58,221.58,18151.42,11,4,0.00,189.57,0.00,2.71
287685,2021-11-14,Kern,California,153681,1733.00,6029,CA,County,900202,17071.83,192.51,192.51,17071.83,14,5,0.00,335.07,0.00,4.57
228570,2021-11-14,San Bernardino,California,367946,5873.00,6071,CA,County,2180085,16877.60,269.39,269.39,16877.60,2,6,0.00,476.93,0.00,6.86
101833,2021-11-14,Stanislaus,California,89701,1387.00,6099,CA,County,550660,16289.73,251.88,251.88,16289.73,4,7,0.00,159.64,0.00,2.14
46374,2021-11-14,Madera,California,25546,293.00,6039,CA,County,157327,16237.52,186.24,186.24,16237.52,17,8,0.00,76.21,0.00,0.43
732829,2021-11-14,Merced,California,44566,645.00,6047,CA,County,277680,16049.41,232.28,232.28,16049.41,8,9,0.00,76.71,0.00,3.79
1049970,2021-11-14,Glenn,California,4524,46.00,6021,CA,County,28393,15933.50,162.01,162.01,15933.50,23,10,0.00,5.86,0.00,0.07


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
460604,2021-11-14,Imperial,California,37398,765.00,6025,CA,County,181215,20637.36,422.15,422.15,20637.36,1,3,0.00,66.93,0.00,0.00
228570,2021-11-14,San Bernardino,California,367946,5873.00,6071,CA,County,2180085,16877.60,269.39,269.39,16877.60,2,6,0.00,476.93,0.00,6.86
3302,2021-11-14,Los Angeles,California,1512028,26862.00,6037,CA,County,10039107,15061.38,267.57,267.57,15061.38,3,14,890.00,1296.71,6.00,16.07
101833,2021-11-14,Stanislaus,California,89701,1387.00,6099,CA,County,550660,16289.73,251.88,251.88,16289.73,4,7,0.00,159.64,0.00,2.14
1113245,2021-11-14,Tuolumne,California,7608,135.00,6109,CA,County,54478,13965.27,247.81,247.81,13965.27,5,19,44.00,22.43,1.00,1.00
80925,2021-11-14,San Joaquin,California,105829,1803.00,6077,CA,County,762148,13885.62,236.57,236.57,13885.62,6,21,0.00,150.21,0.00,2.50
809139,2021-11-14,Inyo,California,2473,42.00,6027,CA,County,18039,13709.19,232.83,232.83,13709.19,7,22,0.00,13.43,0.00,0.07
732829,2021-11-14,Merced,California,44566,645.00,6047,CA,County,277680,16049.41,232.28,232.28,16049.41,8,9,0.00,76.71,0.00,3.79
982400,2021-11-14,Kings,California,34573,347.00,6031,CA,County,152940,22605.60,226.89,226.89,22605.60,9,2,0.00,67.64,0.00,0.71
70454,2021-11-14,Shasta,California,25443,406.00,6089,CA,County,180080,14128.72,225.46,225.46,14128.72,10,18,0.00,65.71,0.00,3.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,51.07
1,03/21/20,Imperial,4,2.21,3,0.00,47.36
2,03/22/20,Imperial,4,2.21,3,0.00,45.21
3,03/23/20,Imperial,4,2.21,3,0.00,40.57
4,03/24/20,Imperial,9,4.97,3,5.00,34.14
...,...,...,...,...,...,...,...
6000,11/10/21,Glenn,4505,15866.59,10,3.00,6.36
6001,11/11/21,Glenn,4513,15894.76,10,8.00,5.57
6002,11/12/21,Glenn,4513,15894.76,10,0.00,5.07
6003,11/13/21,Glenn,4524,15933.50,10,11.00,5.86


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6109,11/10/21,Shasta,402.00,223.23,10,7.00,2.71
6110,11/11/21,Shasta,402.00,223.23,10,0.00,2.71
6111,11/12/21,Shasta,406.00,225.46,10,4.00,3.00
6112,11/13/21,Shasta,406.00,225.46,10,0.00,3.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)